# Ghi chú
(Phiên bản đầu tiên)

In [1]:
'''
CHI CHÚ CÁ NHÂN

Key ảo: 11, 12, 13, 14, 15, ....
Key thực: 0, 1, 2, 3, 4, ....

video 1: 0->12
video 2: 13->25
video 3: 26->38

tên frame (trong cơ sở dữ liệu)
    0, 1, 2, ...
thứ tự frame (so tên frame với map csv)
    0, 100, 271, ...

lưu thứ tự:
    cái lợi là không phải dùng file csv để mapping
lưu theo btc:
    cảm giác quy chuẩn hơn nhưng phải tốn thêm file csv
    dễ tạo file json chung cho toàn bộ data hơn do có thêm file csv để mapping

==> CHỌN LƯU THEO ĐỊNH DẠNG CỦA BTC

database
|__keyframes
|   |__L01_V001
|       |__0001.jpg
|       |__0002.jpg
|       |__...
|
|__map-keyframes
|   |__L01_V001.csv
|
|__kf_info.json
|__index.json
|__index.bin

Thông tin về thời gian xuất hiện của keyframe có lưu trong file csv
theo tên video rồi nhưng mà nếu không lưu chung cho tất cả thì phải
mất thêm thời gian đọc file rồi tìm kiếm, thì tôi nghĩ việc đọc file
có thể mất thời gian. Nếu vậy chi bằng tạo ra một file json load thẳng
lên RAM và viết một class lo việc tạo file json lưu toàn bộ thông tin
và truy xuất thông tin của keyframe đó theo key/id chung của faiss index
, hoặc là để class MyFaiss giống của anh Hùng cũng được, thêm thuộc tính
là một class FrameInfo lấy id, trả về tên video và thời gian xuất hiện
của frame đó trong video (và cả thứ tự frame nếu cần thiết).


Load file index.bin và kf_path.json
Trả về kết quả truy vấn vector id:
    QueryHandler

Xử lý tổng hợp các file csv thành kf_info.json và index.json/kf_path.json
+ tạo sẵn file index.bin
    File4Databse

Load file kf_info.json
+ trả về thông tin frame từ id trong QueryHandler
    FrameInfoHandler
'''

'\nCHI CHÚ CÁ NHÂN\n\nKey ảo: 11, 12, 13, 14, 15, ....\nKey thực: 0, 1, 2, 3, 4, ....\n\nvideo 1: 0->12\nvideo 2: 13->25\nvideo 3: 26->38\n\ntên frame (trong cơ sở dữ liệu)\n    0, 1, 2, ...\nthứ tự frame (so tên frame với map csv)\n    0, 100, 271, ...\n\nlưu thứ tự:\n    cái lợi là không phải dùng file csv để mapping\nlưu theo btc:\n    cảm giác quy chuẩn hơn nhưng phải tốn thêm file csv\n    dễ tạo file json chung cho toàn bộ data hơn do có thêm file csv để mapping\n\n==> CHỌN LƯU THEO ĐỊNH DẠNG CỦA BTC\n\ndatabase\n|__keyframes\n|   |__L01_V001\n|       |__0001.jpg\n|       |__0002.jpg\n|       |__...\n|\n|__map-keyframes\n|   |__L01_V001.csv\n|\n|__kf_info.json\n|__index.json\n|__index.bin\n\nThông tin về thời gian xuất hiện của keyframe có lưu trong file csv\ntheo tên video rồi nhưng mà nếu không lưu chung cho tất cả thì phải\nmất thêm thời gian đọc file rồi tìm kiếm, thì tôi nghĩ việc đọc file\ncó thể mất thời gian. Nếu vậy chi bằng tạo ra một file json load thẳng\nlên RAM và vi

# Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/

Mounted at /content/drive


# Download và import các thư viện

In [3]:
from IPython.display import clear_output

# TẢI CÁC THƯ VIỆN CẦN THIẾT
!pip install faiss-gpu
!pip install pytorch==1.7.1 torchvision cudatoolkit==11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

clear_output()

In [4]:
# IMPORT CÁC THƯ VIỆN CẦN THIẾT
import os
import pandas as pd
import glob
import json
import numpy as np
import faiss
import clip
import torch
from PIL import Image

-----
# Ghi chú Format drive

Class File4Database xử lý tạo các tệp:

    1: kf_info.json: lưu thông tin key/id, thời gianvà tên video
    2: kf_path.json: lưu thông tin key/id và đường dẫn ảnh
    3: index.bin: tạo file index cho truy vấn sau này.

In [5]:
'''
FORMAT DATABASE 1
database/
|__keyframes/
|   |__L01_V001/
|       |__0001.jpg
|       |__0002.jpg
|       |__...
|
|__map-keyframes/
|   |__L01_V001.csv
|   |__...
|
|__kf_info.json
|__kf_path.json
|__index.bin



FORMAT FILE KF_INFO.JSON
{
    <Id>: {
        'video_name': <str>,
        'pts_time': <str>,
        'frame_idx': <str>
    },
    ...
}
Chú thích:
    <Id>        : id chung cho faiss, kiểu <str>
        vd: '1'
    'video_name': tên video chứa keyframe
        vd: 'L01_V001'
    'pts_time'  : thời điểm của keyframe trong video
        vd: '13.4'
    'frame_idx' : thứ tự keyframe trong video
        vd: '335'



FORMAT FILE KF_PATH.JSON
{
    <Id>: <kf_path>,
    ...
}
Chú thích:
    <Id>: giá trị
        vd: '1'
    <kf_path>: đường dẫn ảnh keyframe
        vd: 'keyframes/L01_V001/0001.jpg'
'''
clear_output()

In [6]:
'''
FORMAT DATABASE 2 (Update 20/8/2023)
    database/
    |__keyframes/
    |   |__L01_V001/
    |       |__0001.jpg
    |       |__0002.jpg
    |       |__...
    |
    |__map-keyframes/
    |   |__L01_V001.csv
    |   |__...
    |
    |__npy-keyframes/
    |   |__L01_v001.npy
    |   |__...
    |
    |__kf_info.json
    |__kf_path.json
    |__index.bin


FORMAT DATABASE 3 (Update 4/9/2023)
    KeyFrames_AIO_Pending/
    |__KeyFrames_L01/
    |   |__L01_V001/
    |   |   |__0000.jpg
    |   |   |__0002.jpg
    |   |   |__...
    |   |__L01_V001.txt
    |   |__...
    |
    |__map-keyframes/
    |   |__L01_V001.csv
    |   |__...
    |
    |__npy-keyframes/
    |   |__L01_V001.npy
    |   |__...
    |
    |__kf_info.json
    |__kf_path.json
    |__index.bin
'''
clear_output()

# New code


In [7]:
class File4DB:
    def __init__(self, db_path='/content/drive/MyDrive/KeyFrames_AIO_Pending', quiet=False):
        self.db_path = db_path
        self.kf_folder_path = os.path.join(db_path, 'KeyFrames')
        self.csv_folder_path = os.path.join(db_path, 'map-keyframes')
        self.npy_folder_path = os.path.join(db_path, 'npy-keyframes')

        self.quiet = quiet

        # Mô hình clip cho việc tạo file npy
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model, self.preprocess = clip.load("ViT-L/14@336px", device=self.device)

        if not os.path.exists(self.npy_folder_path):
            os.mkdir(self.npy_folder_path)

    # def __get_video_names(self):
    #     # Trả về list tên các video trong thư mục keyframes
    #     kf_video_paths = sorted(glob.glob(self.kf_folder_path+'/*'))

    #     kf_video_names = [path.split('/')[-1] for path in kf_video_paths]
    #     return kf_video_names

    # def __get_kf_paths(self, video_name):
    #     # Trả về list đường dẫn các ảnh trong video_name trong thư mục keyframes
    #     return sorted(glob.glob(f'{self.kf_folder_path}/{video_name}/*.jpg'))

    def __get_csv_paths(self):
        # Trả về tất cả đường dẫn từng file csv
        return sorted(glob.glob(self.csv_folder_path+'/*.csv'))

    def __get_kf_json_path(self, kf_video_path, pos):
        '''
        kf_video_path: "...KeyFrames_AIO_Pending/Keyframes_L01/L01_V001"
        pos: 0
        return: "KeyFrames_AIO_Pending/KeyFrames_L01/L01_V001/000000.jpg"
        '''
        # giữ lại đường dẫn ví dụ là KeyFrames_AIO_Pending/KeyFrames_L01/L01_V001
        path = kf_video_path.split('/')[-3:]
        path = f'{path[0]}/{path[1]}/{path[2]}'

        # chuyển pos từ kiểu int sang format ví dụ '000000' 6 ký tự
        pos = str(pos)
        n = len(pos)
        for _ in range(6-n):
            pos = '0'+pos

        # tạo đường dẫn ảnh
        path = os.path.join(path, pos+'.jpg')
        return path


    def __get_csv_name(self, csv_path):
        # Ví csv_path = '.../database/map-keyframes/L01_V001.csv'
        # Trả về tên file csv từ một đường dẫn file csv
        csv_name = os.path.split(csv_path)[-1].replace('.csv','')
        csv_group = csv_name[:3]
        return (csv_group, csv_name)

    def __csv_to_df(self, csv_path):
        # Ví csv_path: '.../database/map-keyframes/L01_V001.csv'
        df = pd.read_csv(csv_path)
        csv_group, csv_name = self.__get_csv_name(csv_path)

        # lấy đường dẫn folder keyframe video
        # ví dụ: ".../KeyFrames_AIO_Pending/KeyFrames_L01/L01_V001"
        group_path = self.__get_group_path(csv_group)
        kf_video_path = f'{group_path}/{csv_name}'

        # Tạo thêm 2 cột lưu thông tin đường dẫn key frame và tên video
        df['video_name'] = pd.Series(csv_name, index=np.arange(len(df.index)))
        df['kf_path'] = pd.Series([self.__get_kf_json_path(kf_video_path, i) for i in df['frame_idx']])

        # Bỏ 2 cột thừa 'n' và 'fps'
        df = df.drop(['n', 'fps'], axis=1)
        return df

    # Chuyển kiểu DataFrame sang dạng Json
    def __df_to_json(self, df):
        '''
        df có dạng cột và các dòng cho từng keyframe.
        Hàm sẽ chuyển dataframe lại thành dạng dictionary
        với key là id keyframe và data là thông tin các
        cột tại dòng đó.
        '''
        df = df.astype(str)
        json = {}
        if len(df.columns) == 1:
            # path_json
            df = df.squeeze()
            for i in df.index:
                json[str(i)] = df.iloc[i]
        else:
            # info_json
            for i in df.index:
                info_dict = df.iloc[i].to_dict()
                json[str(i)] = info_dict
        return json


    def create_json(self):
        '''
        Lưu hai file kf_info.json và kf_path.json
        '''
        csv_paths = self.__get_csv_paths()

        # Tạo dataframe rỗng
        columns = ['video_name', 'frame_idx', 'pts_time', 'kf_path']
        index_df = pd.DataFrame(columns=columns)

        for path in csv_paths:
            # đọc file csv và tạo dataframe
            df = self.__csv_to_df(path)

            # Nối dataframe hiện tại với dataframe mới đọc lên
            index_df = pd.concat([index_df, df], ignore_index=True)

        # tách dataframe thành 2 bảng riêng cho 2 file json
        path_df = index_df.loc[:,['kf_path']]
        info_df = index_df.drop(['kf_path'], axis=1)

        # format lại info_df thành dạng json
        path_json = self.__df_to_json(path_df)
        info_json = self.__df_to_json(info_df)

        # Lưu file json
        kf_path_file = self.db_path + '/kf_path.json'
        kf_info_file = self.db_path + '/kf_info.json'
        with open(kf_path_file, 'w') as out_file:
            json.dump(path_json, out_file)
        with open(kf_info_file, 'w') as out_file:
            json.dump(info_json, out_file)


    def __get_group_id(self, video_name):
        # video_name = 'L01_V001'
        return video_name[:3]


    def __get_group_path(self, group_id):
        return f'{self.kf_folder_path}_{group_id}'


    def __get_kf_folder(self,video_name):
        group_id = self.__get_group_id(video_name)
        group_folder = self.__get_group_path(group_id)
        return os.path.join(group_folder, video_name)


    # Hàm tạo một file npy theo tên video
    def __create_npy(self, video_name):
        kf_folder = self.__get_kf_folder(video_name)

        if not self.quiet:
            print(f'Folder path: {kf_folder}')

        # Kiểm tra thư mục có tồn tại hay không
        if not os.path.exists(kf_folder):
            raise Exception(f'Folder {video_name} is not exist!!!')

        img_paths = sorted(glob.glob(kf_folder+'/*.jpg'))

        if len(img_paths) == 0:
            raise Exception(f'Folder {video_name} is empty!!!')

        feats = []
        for i in range(len(img_paths)):
            img = self.preprocess(Image.open(img_paths[i])).unsqueeze(0).to(self.device)
            with torch.no_grad():
                # Tính vector đặc trưng từ ảnh
                feat = self.model.encode_image(img)
            # Chuẩn hóa vector đặc trưng
            feat = feat / feat.norm(dim=-1, keepdim=True)
            feat = feat.detach().cpu().numpy().astype(np.float32).flatten()
            feats.append(feat)
        file_path = f'{self.npy_folder_path}/{video_name}.npy'
        with open(file_path, 'wb') as f:
            np.save(f, np.array(feats, dtype=np.float32))

        if not self.quiet:
            print(f'Creating {video_name}.npy successfully!!!')


    def create_video_npy(self, video_names):
        # video_names = self.__get_video_names()
        for name in video_names:
            self.__create_npy(name)


    def __get_video_names(self, group_id):
        paths = sorted(glob.glob(f'{self.kf_folder_path}_{group_id}/*'))
        txt_paths = sorted(glob.glob(f'{self.kf_folder_path}_{group_id}/*.txt'))
        paths = filter(lambda i: i not in txt_paths, paths)
        names = []
        for path in paths:
            name = os.path.split(path)[-1]
            if len(name) == 8:
                names.append(name)
        return names


    def create_group_npy(self, group_ids):
        for id in group_ids:
            if not self.quiet:
                print(f'Starting create npy files for {id}')
            video_names = self.__get_video_names(id)
            self.create_video_npy(video_names)


    def create_all_npy(self):
        # Lấy groups id từ Keyframes
        group_paths = sorted(glob.glob(f'{self.db_path}/KeyFrames_*'))
        group_ids = []
        for path in group_paths:
            if len(path.split('/')[-1]) == 13:
                group_ids.append(path[-3:])
        self.create_group_npy(group_ids)


    def __read_all_npy(self):
        # Lấy tất cả đường dẫn file npy
        npy_paths = sorted(glob.glob(self.npy_folder_path+'/*.npy'))
        # Xuất hiện thông báo lỗi không tìm thấy file
        if len(npy_paths) == 0:
            raise Exception('Error not found npy files!!!')
        return np.concatenate([np.load(path) for path in npy_paths], axis=0)


    def __create_faiss_bin(self):
        feats = self.__read_all_npy()
        index = faiss.IndexFlatIP(feats.shape[1])
        index.add(feats)
        bin_path = self.db_path+'/index.bin'
        faiss.write_index(index, bin_path)


    def create_bin(self):
        self.__create_faiss_bin()

# Hướng dẫn


In [9]:
'''
# KHỞI TẠO ĐỐI TƯỢNG
f4d = File4DB()

# Tạo file numpy theo tên group
groups = ['L01','L02']
f4d.create_group_npy(groups)
'''
clear_output()

# Chạy tạo các file cần thiết

In [10]:
%%time
# db_path = '.../KeyFrames_AIO_Pending'
# Nếu cần mn có thể sửa lại source mấy cái path
db_path = ''
f4d = File4DB(db_path)

100%|███████████████████████████████████████| 891M/891M [00:16<00:00, 56.7MiB/s]


CPU times: user 11.7 s, sys: 5.9 s, total: 17.7 s
Wall time: 34.9 s


In [ ]:
%%time 

# SỬ DỤNG NẾU CHỈ DÙNG FILE BIN
# f4d.create_json()

# SỬ DỤNG NẾU TẠO FILE BIN LUÔN
f4d.create_all_npy()
f4d.create_json()
f4d.create_bin()

In [ ]:
%%time
# TẠO FILE NUMPY THEO TÊN GROUP
# groups = ['L12']
# f4d.create_group_npy(groups)

# TẠO FILE NUMPY THEO TÊN VIDEO
# videos = ['L11_V0{}'.format(i) for i in range(17,32)]
# f4d.create_video_npy(videos)

In [12]:
%%time
# f4d.create_bin()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
